# Chinle Primary Care Scheduler
**In this notebook, we'll demonstrate how to create an internal medicine schedule for August 2025.**

In [1]:
import sys
from pathlib import Path

# Add project root (one level up from 'tutorial') to sys.path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
import pandas as pd
from datetime import date
from utils.parser import parse_inputs
from utils.calendar import generate_clinic_calendar
from constraints.internal_medicine import (
    create_shift_variables,
    add_leave_constraints,
    add_inpatient_block_constraints,
    add_rdo_constraints,
    add_clinic_count_constraints,
    add_min_max_staffing_constraints
)
from ortools.sat.python import cp_model

**First, we'll import and prepare the YML and CSV files for processing.**

In [3]:
config, leave_df, inpatient_days_df, inpatient_starts_df = parse_inputs('../config/internal_medicine.yml',
                                                                        '../data/leave_requests_july.csv',
                                                                        '../data/inpatient_july.csv')

**Let's examine the parsed YML and CSV data to ensure everything is properly loaded.**

In [4]:
config['clinic_rules']

{'clinic_days': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
 'clinic_sessions': {'Monday': ['morning', 'afternoon'],
  'Tuesday': ['morning', 'afternoon'],
  'Wednesday': ['morning', 'afternoon'],
  'Thursday': ['afternoon'],
  'Friday': ['morning', 'afternoon']},
 'random_day_off': {'eligible_days': ['Monday',
   'Tuesday',
   'Wednesday',
   'Friday']},
 'inpatient_schedule': {'inpatient_length': 7,
  'inpatient_starts_on': 'Tuesday',
  'inpatient_ends_on': 'Monday',
  'pre_inpatient_leave': 'Monday',
  'post_inpatient_leave': 'Friday'},
 'staffing': {'min_providers_per_session': 3, 'max_providers_per_session': 5},
 'holiday_dates': [datetime.date(2025, 1, 1),
  datetime.date(2025, 1, 20),
  datetime.date(2025, 2, 17),
  datetime.date(2025, 5, 26),
  datetime.date(2025, 6, 19),
  datetime.date(2025, 7, 4),
  datetime.date(2025, 9, 1),
  datetime.date(2025, 10, 13),
  datetime.date(2025, 11, 11),
  datetime.date(2025, 11, 27),
  datetime.date(2025, 12, 25)],
 'clinic_int

In [5]:
print(leave_df.dtypes)
leave_df.head(5)

provider            object
date        datetime64[ns]
dtype: object


,provider,date
0,Selig,2025-06-30
1,Selig,2025-07-01
2,Selig,2025-07-02
3,Selig,2025-07-03
4,Selig,2025-07-07


In [6]:
print(inpatient_days_df.dtypes)
inpatient_days_df.head(5)

provider            object
date        datetime64[ns]
dtype: object


,provider,date
0,Orcutt,2025-07-01
1,Orcutt,2025-07-02
2,Orcutt,2025-07-03
3,Orcutt,2025-07-04
4,Orcutt,2025-07-05


In [7]:
print(inpatient_starts_df.dtypes)
inpatient_starts_df.head(5)

provider              object
start_date    datetime64[ns]
dtype: object


,provider,start_date
0,Orcutt,2025-07-01
1,Bornstein,2025-07-01
2,Orcutt,2025-07-22


**The files are correctly formatted with appropriate data types. Now we'll construct our August 2025 calendar, including only valid clinic days (excluding federal holidays) and available sessions (noting there are no Thursday morning sessions).**

In [8]:
calendar = generate_clinic_calendar(date(2025, 6, 30), 
                                    date(2025, 8, 1), 
                                    config['clinic_rules'])

In [9]:
# Here are all the possible clinic sessions that a provider could work in August. 
calendar

{datetime.date(2025, 6, 30): ['morning', 'afternoon'],
 datetime.date(2025, 7, 1): ['morning', 'afternoon'],
 datetime.date(2025, 7, 2): ['morning', 'afternoon'],
 datetime.date(2025, 7, 3): ['afternoon'],
 datetime.date(2025, 7, 7): ['morning', 'afternoon'],
 datetime.date(2025, 7, 8): ['morning', 'afternoon'],
 datetime.date(2025, 7, 9): ['morning', 'afternoon'],
 datetime.date(2025, 7, 10): ['afternoon'],
 datetime.date(2025, 7, 11): ['morning', 'afternoon'],
 datetime.date(2025, 7, 14): ['morning', 'afternoon'],
 datetime.date(2025, 7, 15): ['morning', 'afternoon'],
 datetime.date(2025, 7, 16): ['morning', 'afternoon'],
 datetime.date(2025, 7, 17): ['afternoon'],
 datetime.date(2025, 7, 18): ['morning', 'afternoon'],
 datetime.date(2025, 7, 21): ['morning', 'afternoon'],
 datetime.date(2025, 7, 22): ['morning', 'afternoon'],
 datetime.date(2025, 7, 23): ['morning', 'afternoon'],
 datetime.date(2025, 7, 24): ['afternoon'],
 datetime.date(2025, 7, 25): ['morning', 'afternoon'],
 date

**Next, we'll create a new CpModel instance from the OR-Tools library. This model object will store our scheduling constraints. We'll also generate a dictionary of binary decision variables for each provider, date, and session based on our August 2025 calendar.**

In [10]:
model = cp_model.CpModel()
shift_vars = create_shift_variables(model, list(config['providers'].keys()), calendar)

In [11]:
# For each possible clinic session in August 2025, there's a binary decision variable (0 or 1) that will be solved by the model
shift_vars['Orcutt']

{datetime.date(2025, 6, 30): {'morning': Orcutt_2025-06-30_morning(0..1),
  'afternoon': Orcutt_2025-06-30_afternoon(0..1)},
 datetime.date(2025, 7, 1): {'morning': Orcutt_2025-07-01_morning(0..1),
  'afternoon': Orcutt_2025-07-01_afternoon(0..1)},
 datetime.date(2025, 7, 2): {'morning': Orcutt_2025-07-02_morning(0..1),
  'afternoon': Orcutt_2025-07-02_afternoon(0..1)},
 datetime.date(2025, 7, 3): {'afternoon': Orcutt_2025-07-03_afternoon(0..1)},
 datetime.date(2025, 7, 7): {'morning': Orcutt_2025-07-07_morning(0..1),
  'afternoon': Orcutt_2025-07-07_afternoon(0..1)},
 datetime.date(2025, 7, 8): {'morning': Orcutt_2025-07-08_morning(0..1),
  'afternoon': Orcutt_2025-07-08_afternoon(0..1)},
 datetime.date(2025, 7, 9): {'morning': Orcutt_2025-07-09_morning(0..1),
  'afternoon': Orcutt_2025-07-09_afternoon(0..1)},
 datetime.date(2025, 7, 10): {'afternoon': Orcutt_2025-07-10_afternoon(0..1)},
 datetime.date(2025, 7, 11): {'morning': Orcutt_2025-07-11_morning(0..1),
  'afternoon': Orcutt_20

**With our model initialized, we'll now add the necessary constraints.**

In [12]:
objective_terms = []

In [13]:
add_leave_constraints(model, shift_vars, leave_df)
add_inpatient_block_constraints(model, shift_vars, inpatient_starts_df, inpatient_days_df)
add_clinic_count_constraints(model, shift_vars, config['providers'], inpatient_starts_df)
add_rdo_constraints(model, shift_vars, leave_df, inpatient_days_df, config['clinic_rules'], config['providers'])
add_min_max_staffing_constraints(model, shift_vars, calendar, config['clinic_rules'])

In [14]:
objective_terms.extend(add_clinic_count_constraints(model, shift_vars, config['providers'], inpatient_starts_df))

In [15]:
if objective_terms:
    model.Minimize(sum(objective_terms))

**Finally we solve the model and print the output.**

In [16]:
solver = cp_model.CpSolver()

# Set random seed for reproducibility
solver.parameters.random_seed = 42

status = solver.Solve(model)
solver_wall_time = solver.wall_time

In [17]:
from collections import defaultdict
clinic_rules = config['clinic_rules']

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print(f"Solution found in {solver_wall_time/1000:.6f} seconds") # Convert from ms to seconds
    
    # Track sessions by provider and week
    provider_sessions = defaultdict(lambda: defaultdict(int))
    
    # Print the schedule and collect session counts
    for day in calendar:
        day_of_week = day.strftime('%A')
        week_key = (day.year, day.isocalendar()[1])
        
        for session in calendar[day]:
            scheduled = [
                provider for provider in shift_vars
                if day in shift_vars[provider]
                and session in shift_vars[provider][day]
                and solver.Value(shift_vars[provider][day][session]) == 1
            ]
            
            # Update session counts for each provider
            for provider in scheduled:
                provider_sessions[provider][week_key] += 1
            
            print(f"{day} {day_of_week} {session}: staffed by {scheduled}")
    
    # Print session counts per provider per week
    print("\n=== Provider Weekly Session Counts ===")
    all_providers = list(config['providers'])
    all_weeks = sorted(set(week for provider_weeks in provider_sessions.values() 
                          for week in provider_weeks))
    
    # Header row with week numbers
    header = "Provider    " + "".join(f"Week {week[1]:02d}  " for week in all_weeks)
    print(header)
    print("-" * len(header))
    
    # Print counts for each provider
    for provider in all_providers:
        row = f"{provider:<10} " + "".join(
            f"{provider_sessions[provider][week]:^10}" for week in all_weeks
        )
        print(row)
else:
    print("No feasible solution found.")

Solution found in 0.000015 seconds
2025-06-30 Monday morning: staffed by ['Miles', 'Orcutt', 'Stewart', 'Tanay', 'Wadlin']
2025-06-30 Monday afternoon: staffed by ['Bornstein', 'Miles', 'Orcutt', 'Tanay', 'Wadlin']
2025-07-01 Tuesday morning: staffed by ['Bornstein', 'Mcrae', 'Orcutt', 'Tanay', 'Wadlin']
2025-07-01 Tuesday afternoon: staffed by ['Bornstein', 'Mcrae', 'Orcutt', 'Stewart', 'Tanay']
2025-07-02 Wednesday morning: staffed by ['Mcrae', 'Orcutt', 'Stewart']
2025-07-02 Wednesday afternoon: staffed by ['Bornstein', 'Mcrae', 'Stewart', 'Wadlin']
2025-07-03 Thursday afternoon: staffed by ['Mcrae', 'Miles', 'Tanay', 'Wadlin']
2025-07-07 Monday morning: staffed by ['Bornstein', 'Mcrae', 'Orcutt']
2025-07-07 Monday afternoon: staffed by ['Mcrae', 'Miles', 'Orcutt']
2025-07-08 Tuesday morning: staffed by ['Selig', 'Tanay', 'Wadlin']
2025-07-08 Tuesday afternoon: staffed by ['Selig', 'Stewart', 'Tanay']
2025-07-09 Wednesday morning: staffed by ['Selig', 'Stewart', 'Tanay', 'Wadlin']
2